 To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [15]:
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Wed, 09 Oct 2019 10:28:12 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '14973', 'Connection': 'keep-alive', 'Server': 'mw1249.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.22-1+0~20190902.26+debian9~1.gbpd64eb7+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Mon, 23 Sep 2019 14:55:44 GMT', 'Backend-Timing': 'D=101517 t=1570460144346446', 'Content-Encoding': 'gzip', 'X-Varnish': '225776502 251801884, 913721586 132514133, 294316039 619414934', 'Age': '156746', 'X-Cache': 'cp1081 hit/7, cp3030 hit/3, cp3041 hit/13', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=09-Oct-2019;Path=/;Http

In [17]:
web_soup = BeautifulSoup(result.content, 'html.parser')
table_str = web_soup.find('table')
alltrs = table_str.find_all('tr')
rows = []
for tr in alltrs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
Rec = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        Rec.append([postalcode, borough, neighborhood])

# Rec

In [18]:
cols = ['PostalCode', 'Borough', 'Neighborhood'] #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(Rec, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


In [19]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page,
# you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
# These two rows will be combined into one row with the neighborhoods separated with a comma
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [20]:
df.loc[df['PostalCode'] == 'M5A']


,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [21]:

df.shape

(103, 3)

In [22]:
dfgeo = pd.read_csv("C:/Users/P7576/Desktop/Learning on Cousera/Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [23]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')


In [24]:
df2.loc[df2['PostalCode'] == 'M5G']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [25]:

df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [26]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [28]:
CLIENT_ID = 'GSKMHCVTNZKPLYCE3KQMFY5TOKALWAEODC4IQCNPQW0RJKNT' # your Foursquare ID
CLIENT_SECRET = 'KSLZQ2INRQFXSSAEAFV34VKAXHPHA3SPLBOYL5RDTQRAQD5C' # your Foursquare Secret
VERSION = '20180605'

In [29]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GSKMHCVTNZKPLYCE3KQMFY5TOKALWAEODC4IQCNPQW0RJKNT&client_secret=KSLZQ2INRQFXSSAEAFV34VKAXHPHA3SPLBOYL5RDTQRAQD5C&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [36]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [37]:
print(venues.shape)
venues.head()

(2263, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [38]:

venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,111,111,111,111,111,111
Downtown Toronto,1295,1295,1295,1295,1295,1295
East Toronto,127,127,127,127,127,127
East York,78,78,78,78,78,78
Etobicoke,81,81,81,81,81,81
Mississauga,11,11,11,11,11,11
North York,240,240,240,240,240,240
Queen's Park,37,37,37,37,37,37
Scarborough,87,87,87,87,87,87


In [39]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 276 unique categories.


In [40]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:

onehot.shape

(2263, 276)

In [42]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018018,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,Downtown Toronto,0.000000,0.000772,0.000772,0.000772,0.000772,0.001544,0.001544,0.000772,0.013900,...,0.002317,0.013900,0.002317,0.000000,0.004633,0.000000,0.006950,0.000772,0.000000,0.003089
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023622,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023622
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004167,0.000000,0.004167,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,...,0.000000,0.000000,0.004167,0.004167,0.008333,0.000000,0.000000,0.004167,0.020833,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.027027
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.005618,0.000000,0.000000,0.011236,0.000000,0.005618,0.000000,0.000000,0.005618


In [43]:
grouped.shape

(11, 276)

In [44]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3     Pizza Place  0.05
4            Park  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.05
2  Italian Restaurant  0.03
3               Hotel  0.03
4          Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4                Café  0.04


----East York----
          venue  freq
0   Coffee Shop  0.08
1  Burger Joint  0.05
2   Pizza Place  0.04
3          Park  0.04
4          Bank  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.06
2        Pharmacy  0.05
3     Coffee Shop  0.05
4            Park  0.04


----Mississauga----
                      venue  freq
0                     Hotel  0.18
1               Coffee Shop  0.18
2     

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Gym,Dessert Shop,Sushi Restaurant,Pub,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Japanese Restaurant,Park,Steakhouse
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Pizza Place,Bakery,Park,Yoga Studio
3,East York,Coffee Shop,Burger Joint,Sporting Goods Shop,Pizza Place,Park,Bank,Pharmacy,Liquor Store,Supermarket,Athletics & Sports
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Liquor Store,Discount Store,Café,Fast Food Restaurant,Grocery Store,Gym
5,Mississauga,Coffee Shop,Hotel,Gym / Fitness Center,Fried Chicken Joint,Middle Eastern Restaurant,Mediterranean Restaurant,Sandwich Place,American Restaurant,Burrito Place,Donut Shop
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Restaurant,Japanese Restaurant,Park,Café,Grocery Store,Sandwich Place
7,Queen's Park,Coffee Shop,Gym,Park,Yoga Studio,Diner,Italian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
8,Scarborough,Breakfast Spot,Coffee Shop,Fast Food Restaurant,Bakery,Chinese Restaurant,Pizza Place,Pharmacy,Intersection,Thai Restaurant,Bank
9,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Breakfast Spot,Bookstore,Men's Store,Pizza Place


In [47]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 4, 1, 0, 3, 4, 0])

In [48]:

merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018018,...,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Gym,Dessert Shop,Sushi Restaurant,Pub,Clothing Store
1,Downtown Toronto,0.0,0.000772,0.000772,0.000772,0.000772,0.001544,0.001544,0.000772,0.013900,...,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Japanese Restaurant,Park,Steakhouse
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023622,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Pizza Place,Bakery,Park,Yoga Studio
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Burger Joint,Sporting Goods Shop,Pizza Place,Park,Bank,Pharmacy,Liquor Store,Supermarket,Athletics & Sports
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,...,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Liquor Store,Discount Store,Café,Fast Food Restaurant,Grocery Store,Gym
